## NICAR 2020: NLP Workshop

Spacy é uma biblioteca que pode ajudá-lo a fazer análises linguísticas.

Para instalar e usar a versão em inglês do spacy, você deve executar estes comandos em seu ambiente virtual:
`pip3 install spacy`
`python3 -m spacy download en_core_web_sm`
Nós estaremos importando o arquivo `text.txt` que está no sua pasta `data`. Ele contém um artigo de amostra sobre um artigo muito especial [cat](https://www.buzzfeednews.com/article/juliareinstein/this-thicc-lazy-high-maintenance-incredibly-well-hydrated/).

Para o português você deve instalar: `python -m spacy download pt_core_news_sm`

In [2]:
# Importar nossas bibliotecas
import spacy
import pandas as pd

In [5]:
# Carregar English tokenizer, tagger, parser, NER e vetores de palavras
nlp = spacy.load('en_core_web_sm')

# Para analisar a língua portuguesa seria feito isto ao invés
# nlp = spacy.load('pt_core_news_sm')

# abre o arquivo de texto e o transforma em uma string
text = open("../data/text.txt","r+").read()
len(text) # isso retorna o comprimento de caracteres e espaços

2990

### Corpus & Tokenizing

Agora vamos transformar o string em um corpus para spacy!

In [6]:
doc = nlp(text)

O documento pode funcionar como uma lista de palavras. Para acessar cada palavra ou 'token', podemos usar a função incorporada `.text`

In [ ]:
for token in doc:
    print(token.text)

Agora podemos contar algumas palavras:
- transformando as palavras em uma lista
- transformando essa lista em um dataframe de pandas
- contando os valores

In [8]:
rows = []
for token in doc:
    rows.append(token.text)

In [ ]:
print(rows)

In [10]:
word_dataframe = pd.DataFrame(rows)
word_dataframe.columns = ['word']
word_dataframe.head()

,word
0,This
1,is
2,Bruno
3,","
4,and


Duas maneiras de contar:

In [11]:
word_count = word_dataframe['word'].value_counts().reset_index()
word_count.head()

,index,word
0,",",34
1,.,33
2,I,20
3,\n\n,18
4,the,18


In [12]:
word_count_alt = word_dataframe.groupby('word').agg({"word":"count"})
word_count_alt.head()

,word
word,
\n,1
\n\n,18
!,1
"""",12
's,1


Você pode salvar as saídas usando `.to_csv`:

In [13]:
word_count.to_csv('../output/word_count.csv', index=False)
word_count_alt.to_csv('../output/word_count2.csv')

### Mais maneiras de filtrar e limpar

Quando você converte um documento em tokens usando o spacy, ele não contém apenas informações sobre cada palavra. 

#### Stopwords

Spacy tem uma lista interna de 'stopwords', ou palavras extremamente comuns que podem não adicionar muita percepção à nossa análise. Quando você tokeniza seu documento, ele também verifica seus tokens com as palavras de parada.

In [14]:
rows = []
for token in doc:
    rows.append([token.text, token.is_stop])

In [15]:
stop_dataframe = pd.DataFrame(rows)
stop_dataframe.columns = ['word', 'is_stop']
stop_dataframe.head()

,word,is_stop
0,This,True
1,is,True
2,Bruno,False
3,",",False
4,and,True


Compare a diferença entre as contagens agregadas com e sem as stopwords:

In [16]:
no_stop = stop_dataframe[~stop_dataframe['is_stop']]

no_stop['word'].value_counts().head(10)

,          34
.          33
\n\n       18
“          12
"          12
”          12
said        8
shelter     7
Bruno       6
foster      6
Name: word, dtype: int64

In [17]:
stop_dataframe['word'].value_counts().head(10)

,       34
.       33
I       20
\n\n    18
the     18
a       15
to      15
my      13
and     12
“       12
Name: word, dtype: int64

#### Lematização

Quando você tokeniza seu documento com spacy, ele também _lemmatizes_ este, ou agrupa palavras e seus derivados (e.g., _organize_, _organized_, and _organizing_). Por exemplo:

- am, are, is $\Rightarrow$ be
- car, cars, car's, cars' $\Rightarrow$ car

Você pode acessar a palavra lematizada de um token usando o `token.lemma_`.

In [18]:
rows = []
for token in doc:
    rows.append([token.text, token.lemma_])

In [19]:
lemma_dataframe = pd.DataFrame(rows)
lemma_dataframe.columns = ['word', 'lemma']
lemma_dataframe.head(10)

,word,lemma
0,This,this
1,is,be
2,Bruno,Bruno
3,",",","
4,and,and
5,he,-PRON-
6,’s,’
7,a,a
8,25-pound,25-pound
9,cat,cat


Compare a diferença entre as contagens agregadas com e sem as stopwords:

In [20]:
lemma_dataframe['lemma'].value_counts()

-PRON-      84
"           36
,           34
.           33
be          28
            ..
weekend      1
BuzzFeed     1
live         1
face         1
:            1
Name: lemma, Length: 243, dtype: int64

In [21]:
lemma_dataframe['word'].value_counts()

,             34
.             33
I             20
\n\n          18
the           18
              ..
toys           1
live           1
particular     1
become         1
:              1
Name: word, Length: 288, dtype: int64

### Bonus: Extraindo entidades

Uma tarefa comum da PNL que pode ser útil é a extração de **entities**. Isso inclui pessoas, negócios, locais, organizações e datas. Para obter uma lista completa dos tipos de entidade que a spacy é capaz de reconhecer imediatamente, você pode consultar a [documentação](https://spacy.io/api/annotation#named-entities).

In [ ]:
print(rows)

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Você pode vê-los destacados no texto usando `displacy`.

In [24]:
from spacy import displacy
displacy.render(doc, style = "ent",jupyter = True)

### O mesmo em português

In [1]:
import spacy
import pandas as pd

In [2]:
nlp = spacy.load('pt_core_news_sm')

In [4]:
texto = "Macunaíma, herói de nossa gente nasceu à margem do Uraricoera, em plena floresta amazônica. Descendia da tribo dos Tapanhumas e, desde a primeira infância, revelava-se como um sujeito preguiçoso. Ainda menino, busca prazeres amorosos com Sofará, mulher de seu irmão Jiguê, que só lhe havia dado pra comer as tripas de uma anta, caçada por Macunaíma numa armadilha esperta. Nas várias transas (brincadeiras) com Sofará, Macunaíma transforma-se num príncipe lindo, iniciando um processo constante de metamorfoses que irão ocorrer ao longo da narrativa: índio negro, vira branco, inseto, peixe e até mesmo um pato, dependendo das circunstâncias."

In [5]:
len(texto)

642

In [6]:
doc = nlp(texto)

In [7]:
for token in doc:
    print(token.text)

Macunaíma
,
herói
de
nossa
gente
nasceu
à
margem
do
Uraricoera
,
em
plena
floresta
amazônica
.
Descendia
da
tribo
dos
Tapanhumas
e
,
desde
a
primeira
infância
,
revelava-se
como
um
sujeito
preguiçoso
.
Ainda
menino
,
busca
prazeres
amorosos
com
Sofará
,
mulher
de
seu
irmão
Jiguê
,
que
só
lhe
havia
dado
pra
comer
as
tripas
de
uma
anta
,
caçada
por
Macunaíma
numa
armadilha
esperta
.
Nas
várias
transas
(
brincadeiras
)
com
Sofará
,
Macunaíma
transforma-se
num
príncipe
lindo
,
iniciando
um
processo
constante
de
metamorfoses
que
irão
ocorrer
a
o
longo
da
narrativa
:
índio
negro
,
vira
branco
,
inseto
,
peixe
e
até
mesmo
um
pato
,
dependendo
das
circunstâncias
.


In [8]:
rows = []
for token in doc:
    rows.append(token.text)

In [9]:
print(rows)

['Macunaíma', ',', 'herói', 'de', 'nossa', 'gente', 'nasceu', 'à', 'margem', 'do', 'Uraricoera', ',', 'em', 'plena', 'floresta', 'amazônica', '.', 'Descendia', 'da', 'tribo', 'dos', 'Tapanhumas', 'e', ',', 'desde', 'a', 'primeira', 'infância', ',', 'revelava-se', 'como', 'um', 'sujeito', 'preguiçoso', '.', 'Ainda', 'menino', ',', 'busca', 'prazeres', 'amorosos', 'com', 'Sofará', ',', 'mulher', 'de', 'seu', 'irmão', 'Jiguê', ',', 'que', 'só', 'lhe', 'havia', 'dado', 'pra', 'comer', 'as', 'tripas', 'de', 'uma', 'anta', ',', 'caçada', 'por', 'Macunaíma', 'numa', 'armadilha', 'esperta', '.', 'Nas', 'várias', 'transas', '(', 'brincadeiras', ')', 'com', 'Sofará', ',', 'Macunaíma', 'transforma-se', 'num', 'príncipe', 'lindo', ',', 'iniciando', 'um', 'processo', 'constante', 'de', 'metamorfoses', 'que', 'irão', 'ocorrer', 'a', 'o', 'longo', 'da', 'narrativa', ':', 'índio', 'negro', ',', 'vira', 'branco', ',', 'inseto', ',', 'peixe', 'e', 'até', 'mesmo', 'um', 'pato', ',', 'dependendo', 'das', 

In [10]:
word_dataframe = pd.DataFrame(rows)
word_dataframe.columns = ['word']
word_dataframe.head()

,word
0,Macunaíma
1,","
2,herói
3,de
4,nossa


In [11]:
word_count = word_dataframe['word'].value_counts().reset_index()
word_count.head()

,index,word
0,",",14
1,de,4
2,.,4
3,Macunaíma,3
4,um,3


In [12]:
word_count_alt = word_dataframe.groupby('word').agg({"word":"count"})
word_count_alt.head()

,word
word,
(,1
),1
",",14
.,4
:,1


In [13]:
word_count.to_csv('../output/macunaima_word_count.csv', index=False)
word_count_alt.to_csv('../output/macunaima_word_count2.csv')

In [14]:
rows = []
for token in doc:
    rows.append([token.text, token.is_stop])

In [15]:
stop_dataframe = pd.DataFrame(rows)
stop_dataframe.columns = ['word', 'is_stop']
stop_dataframe.head()

,word,is_stop
0,Macunaíma,False
1,",",False
2,herói,False
3,de,True
4,nossa,True


In [16]:
no_stop = stop_dataframe[~stop_dataframe['is_stop']]

no_stop['word'].value_counts().head(10)

,             14
.              4
Macunaíma      3
e              2
Sofará         2
a              2
iniciando      1
(              1
mulher         1
dependendo     1
Name: word, dtype: int64

In [17]:
stop_dataframe['word'].value_counts().head(10)

,            14
de            4
.             4
Macunaíma     3
um            3
a             2
que           2
Sofará        2
da            2
e             2
Name: word, dtype: int64

In [18]:
rows = []
for token in doc:
    rows.append([token.text, token.lemma_])

In [19]:
lemma_dataframe = pd.DataFrame(rows)
lemma_dataframe.columns = ['word', 'lemma']
lemma_dataframe.head(10)

,word,lemma
0,Macunaíma,Macunaíma
1,",",","
2,herói,herói
3,de,de
4,nossa,nosso
5,gente,gente
6,nasceu,nascer
7,à,à
8,margem,margem
9,do,do


In [20]:
lemma_dataframe['lemma'].value_counts()

,              14
de              4
o               4
.               4
um              3
               ..
florestar       1
primeiro        1
metamorfose     1
dar             1
haver           1
Name: lemma, Length: 87, dtype: int64

In [21]:
lemma_dataframe['word'].value_counts()

,            14
de            4
.             4
Macunaíma     3
um            3
             ..
as            1
herói         1
processo      1
longo         1
príncipe      1
Name: word, Length: 90, dtype: int64

In [22]:
print(rows)

[['Macunaíma', 'Macunaíma'], [',', ','], ['herói', 'herói'], ['de', 'de'], ['nossa', 'nosso'], ['gente', 'gente'], ['nasceu', 'nascer'], ['à', 'à'], ['margem', 'margem'], ['do', 'do'], ['Uraricoera', 'Uraricoera'], [',', ','], ['em', 'em'], ['plena', 'pleno'], ['floresta', 'florestar'], ['amazônica', 'amazônica'], ['.', '.'], ['Descendia', 'Descendia'], ['da', 'da'], ['tribo', 'tribo'], ['dos', 'dos'], ['Tapanhumas', 'Tapanhumas'], ['e', 'e'], [',', ','], ['desde', 'desde'], ['a', 'o'], ['primeira', 'primeiro'], ['infância', 'infância'], [',', ','], ['revelava-se', 'revelava-se'], ['como', 'comer'], ['um', 'um'], ['sujeito', 'sujeitar'], ['preguiçoso', 'preguiçoso'], ['.', '.'], ['Ainda', 'Ainda'], ['menino', 'menino'], [',', ','], ['busca', 'buscar'], ['prazeres', 'prazer'], ['amorosos', 'amoroso'], ['com', 'com'], ['Sofará', 'Sofará'], [',', ','], ['mulher', 'mulher'], ['de', 'de'], ['seu', 'seu'], ['irmão', 'irmão'], ['Jiguê', 'Jiguê'], [',', ','], ['que', 'que'], ['só', 'só'], ['lh

In [23]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Macunaíma ORG
Uraricoera LOC
Tapanhumas LOC
Sofará LOC
Jiguê PER
Macunaíma PER
Sofará LOC
Macunaíma LOC


In [24]:
from spacy import displacy
displacy.render(doc, style = "ent",jupyter = True)

In [25]:
# Observação - a detectação de entidades tem falhas em português, principalmente em um texto mais antigo como este